# Linked List Implementation in Rust

Recursive definition of a singly linked list:

```
List a = Empty | Elem a (List a)
```

In [2]:
enum List {
    Empty,
    Elem(i32, List)
}

Error: recursive type `List` has infinite size

Because Rust does not support direct recursive types, we use a `Box` to enable heap allocation for the recursive part of the list:

In [3]:
#[derive(Debug)]
enum List<T> {
    Elem(T, Box<List<T>>),
    Empty
}

In [4]:
let list: List<i32> = List::Elem(1, Box::new(List::Elem(2, Box::new(List::Empty))));
list

Elem(1, Elem(2, Empty))

This implementation is not optimal.

Consider a list with two elements:

```
[] = Stack
() = Heap

[Elem 1, ptr] -> (Elem 2, ptr) -> (Empty)
```
What happens here is that the first element of the list is stored on the stack, while the second element and the `Empty` variant are stored on the heap. We heap allocate the `Empty` node. On the other hand, we could allocate the first element on the heap. List should look like this:

```
[ptr] -> (Elem 1, ptr) -> (Elem 2, Empty)
```

## The better list implementation

We can use `Option` to represent the link to the next node. This way, we can avoid heap allocating the `Empty` node.

In [2]:
#[derive(Debug)]
struct Node<T> {
    elem: T,
    next: Link<T>
}

type Link<T> = Option<Box<Node<T>>>;

#[derive(Debug)]
pub struct List<T> {
    head: Link<T>
}

## Implementing new, push, and pop methods

In [3]:
impl<T> List<T> {
    pub fn new() -> Self {
        List { head: None }
    }

    pub fn push(&mut self, elem: T) {
        let new_node = Box::new(Node {
            elem, 
            next: self.head
        });

        self.head = Some(new_node)
    } 
}

Error: cannot move out of `self.head` which is behind a mutable reference

### We cannot move out of borrowed content

In Rust we cannot move out of borrowed content. This means that if we have a reference to a value, we cannot move the value out of the reference. This is important for our linked list implementation because we want to be able to traverse the list without moving the nodes out of the list. 

**Borrowing Guarantees**

When you borrow a value (either immutably or mutably), Rust guarantees:

* The **owner** of the value retains control of the memory.
* The **borrower** can access the value, but **not take ownership** of it.
* The **original value must remain valid** for the duration of the borrow.

If you were allowed to move out of a borrowed value

* The original owner would be left with a **partially moved-from** value.
* This would **invalidate the borrow**, breaking Rust’s safety guarantees.
* It could lead to **use-after-free**, **double-free**, or **undefined behavior**.


We can use `std::mem::replace` to work around this limitation. This function allows us to replace the value at a mutable reference with a new value, returning the old value. This way, we can effectively "move" the value out of the reference without violating Rust's borrowing rules.

In [4]:
impl<T> List<T> {
    pub fn new() -> Self {
        List { head: None }
    }

    pub fn push(&mut self, elem: T) {
        let new_node = Box::new(Node {
            elem, 
            next: std::mem::replace(&mut self.head, None) // we are moving out by replacing
        });

        self.head = Some(new_node)
    } 
}

In [6]:
let mut lst = List::<i32>::new();
lst.push(42);
lst.push(665);

lst

List { head: Some(Node { elem: 665, next: Some(Node { elem: 42, next: None }) }) }

In [7]:
impl<T> List<T> {
    pub fn pop(&mut self) -> Option<T> {
        match std::mem::replace(&mut self.head, None) {
            None => None,
            Some(node) => {
                self.head = node.next;
                Some(node.elem)
            }
        }
    }
}

In [8]:
let value = lst.pop();
println!("popped value: {value:?}");
println!("lst after pop: {lst:?}");

popped value: Some(665)
lst after pop: List { head: Some(Node { elem: 42, next: None }) }


In [9]:
mod tests {
    use super::*;

    pub fn basic_operations_on_list() {
        let mut lst = List::<i32>::new();

        assert_eq!(lst.pop(), None);

        // populate list
        lst.push(1);
        lst.push(2);
        lst.push(3);

        // normal removal
        assert_eq!(lst.pop(), Some(3));
        assert_eq!(lst.pop(), Some(2));
        
        // Push some more just to make sure nothing's corrupted
        lst.push(4);
        lst.push(5);

        // Check normal removal
        assert_eq!(lst.pop(), Some(5));
        assert_eq!(lst.pop(), Some(4));

        // Check exhaustion
        assert_eq!(lst.pop(), Some(1));
        assert_eq!(lst.pop(), None);
    }
}

tests::basic_operations_on_list();

### Dropping the list

When we drop the list, Rust will automatically drop each node in the list recursively. This is because each node owns its next node via the `Box`, and when a `Box` is dropped, it drops the value it points to. Thus, dropping the head of the list will trigger a chain reaction that drops all nodes in the list. It can overflow the stack for very long lists.

The solution is to implement an iterative `drop` method for the list that traverses the list and drops each node one by one, avoiding deep recursion.

In [10]:
impl<T> Drop for List<T> {
    fn drop(&mut self) {
        let mut current_link = std::mem::replace(&mut self.head, None);

        while let Some(mut boxed_node) = current_link { 
            current_link = std::mem::replace(&mut boxed_node.next, None);
            // boxed_node goes out of scope and gets dropped here
            // but its Node's `next` field has been set to None => so no unbounded recursion occurs
        }
   }
}

## Implement peek method

In [12]:
impl<T> List<T> {
    pub fn peek(&self) -> Option<&T> {
        self.head.as_ref().map(|node| { &node.elem })
    }

    pub fn peek_mut(&mut self) -> Option<&mut T> {
        self.head.as_mut().map(|node| { &mut node.elem })
    }
}

In [15]:
mod tests {
    use super::*;

    pub fn peek_elements_on_the_list() {
        let mut lst = List::new();

        assert_eq!(lst.peek(), None);
        assert_eq!(lst.peek_mut(), None);

        lst.push(1);
        assert_eq!(lst.peek(), Some(&1));
        assert_eq!(lst.peek_mut(), Some(&mut 1));

        lst.peek_mut().map(|value| { *value = 42; });
        assert_eq!(lst.pop(), Some(42));
        assert_eq!(lst.peek(), None);
    }
}

tests::peek_elements_on_the_list();

## Implementing Iterators

Iterator is a trait that allows you to define how to iterate over a collection. It requires you to implement the `next` method, which returns the next item in the iteration or `None` if there are no more items.

In [ ]:
mod Explain {
    pub trait Iterator {
        type Item;

        fn next(&mut self) -> Option<Self::Item>;
    }
}

We have three iterator types to implement for our linked list:
* `IntoIter` - takes ownership of the list and yields elements by value.
* `Iter` - borrows the list immutably and yields references to the elements.
* `IterMut` - borrows the list mutably and yields mutable references to the elements.

### IntoIter Implementation

In [18]:
pub struct IntoIter<T>(List<T>); // tuple struct with a single field of type List<T>

impl<T> List<T> {
    pub fn into_iter(self) -> IntoIter<T> {
        IntoIter(self)
    }
}

impl<T> Iterator for IntoIter<T> {
    type Item = T;

    fn next(&mut self) -> Option<Self::Item> {
        self.0.pop()
    }
}

In [19]:
mod tests {
    use super::*;

    pub fn into_iter() {
        let mut lst = List::new();

        lst.push(1); lst.push(2); lst.push(3);

        let mut iter = lst.into_iter();
        assert_eq!(iter.next(), Some(3));
        assert_eq!(iter.next(), Some(2));
        assert_eq!(iter.next(), Some(1));
        assert_eq!(iter.next(), None);
    }
}

tests::into_iter();

### Iter Implementation

In [43]:
// Iter is generic over *some* lifetime called 'a and type T
pub struct Iter<'a, T> {
    next: Option<&'a Node<T>>
}

// No lifetime annotations needed - List owns its data
impl<T> List<T> {
    // We declare a fresh lifetime 'a for this method that borrows self creates the Iter struct.
    // Now &self needs to be valid as long as the Iter struct is used.
    pub fn iter(&self) -> Iter<'_, T> {
        // Iter { next: self.head.as_ref().map(|node| &**node) }
        Iter { next: self.head.as_deref() } // as_deref converts Option<Box<Node<T>>> to Option<&Node<T>>
    }
}   

// Now we implement Iterator for Iter, specifying the same lifetime 'a
impl<'a, T> Iterator for Iter<'a, T> {
    type Item = &'a T;

    fn next(&mut self) -> Option<Self::Item> {
        self.next.map(|node| {
            // self.next = node.next.as_ref().map(|next_node| &**next_node);
            self.next = node.next.as_deref();
            &node.elem
        })
    }
}

### Lifetimes in Rust

A lifetime is the name of a region (~block/scope) of code somewhere in a program. That's it. When a reference is tagged with a lifetime, we're saying that it has to be valid for that entire region.

For most cases, Rust can infer the lifetimes of references on its own.

```rust
// Only one reference in input, so the output must be derived from that input
fn foo(&A) -> &B; // sugar for:
fn foo<'a>(&'a A) -> &'a B;

// Many inputs, assume they're all independent
fn foo(&A, &B, &C); // sugar for:
fn foo<'a, 'b, 'c>(&'a A, &'b B, &'c C);

// Methods, assume all output lifetimes are derived from `self`
fn foo(&self, &B, &C) -> &D; // sugar for:
fn foo<'a, 'b, 'c>(&'a self, &'b B, &'c C) -> &'a D;
``` 

However, in some cases, we need to explicitly annotate lifetimes to help the compiler understand how long references should be valid.

In [47]:
mod tests {
    use super::*;

    pub fn iter() {
        let mut lst = List::new();

        lst.push(1); lst.push(2); lst.push(3);

        let mut iter = lst.iter();
        assert_eq!(iter.next(), Some(&3));
        assert_eq!(iter.next(), Some(&2));
        assert_eq!(iter.next(), Some(&1));
        assert_eq!(iter.next(), None);
    }
}

tests::iter();

### IterMut Implementation

In [45]:
pub struct IterMut<'a, T> {
    next: Option<&'a mut Node<T>>
}

impl<T> List<T> {
    pub fn iter_mut(&mut self) -> IterMut<'_, T> {
        IterMut { next: self.head.as_deref_mut() }
    }
}

impl<'a, T> Iterator for IterMut<'a, T> {
    type Item = &'a mut T;

    fn next(&mut self) -> Option<Self::Item> {
        self.next.take().map(|node| {
            self.next = node.next.as_deref_mut();
            &mut node.elem
        })
    }
}

In [46]:
mod tests {
    use super::*;

    pub fn iter_mut() {
        let mut list = List::new();
        list.push(1); list.push(2); list.push(3);

        let mut iter = list.iter_mut();
        assert_eq!(iter.next(), Some(&mut 3));
        assert_eq!(iter.next(), Some(&mut 2));
        assert_eq!(iter.next(), Some(&mut 1));
        }
}

tests::iter_mut();